In [12]:
import numpy as np
import os
import glob
import pickle
import music21 as m21
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, stream, note, chord


# First Create The Note Object From Midi Files

In [7]:
def notes_from_midi():
    #create a note objects off all notes, chords, and rests from midi files
    notes = []
    
    for file in glob.glob('midi_files/*.mid'):
        midi = converter.parse(file)
        
        #print(midi) -> To make sure it is creating a music stream object
        
        #notes to parse
        notes_to_parse = None
        
        try: #songs have multiple paino parts
            p_parts = instrument.partitionByInstrument(midi)
#             print(len(p_parts)) # just checking to see if the output is what I want
#             print(p_parts[1])
            notes_to_parse = p_parts[0].recurse()
        except: #when note is not a chord
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " + str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name) + " " + str(element.quarterLength))
                
#run this when you are ready to save the notes make sure they are correct first.
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes        

In [3]:
def prepare_notes():
    '''
    prepare the notes to be the input and output used by the network
    
    notes = note object created after parsing the midi files using M21
    
    Output: The input and output sequences to the LSTM network
    
    '''
    
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 100 
    
    #creating all the unique notes for create the dictionary from
    pitches = sorted(set(note for note in notes))
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_input = lstm_input / float(len(pitches))
    #one_hot_encoding lstm_ input
    lstm_output = to_categorical(lstm_output)
    
    return lstm_input, lstm_output

In [5]:
lstm_input, lstm_output = prepare_notes()

(68473, 2424)

In [21]:
print (lstm_input.shape)

(68473, 100, 1)


In [8]:
notes = pickle.load(pickle_file)

NameError: name 'pickle_file' is not defined

In [10]:
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)

In [11]:
notes

['rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.50.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.00.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.50.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.00.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.50.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.00.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '7.110.5',
 'F3 0.5',
 'C5 0.5',
 'C4 0.5',
 'D5 0.5',
 'rest 0.5',
 '5.9.02.25',
 'C4 0.5',
 'rest 0.5',
 'C4 0.5',
 'rest 0.5',
 'C4 0.5',
 'rest 1.25',
 'F3 0.5',
 'C4 0.5',
 '4.90.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'E5 0.25',
 '2.50.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'D5 0.25',
 '9.00.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'C5 0.25',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C

In [13]:
pitches = sorted(set(note for note in notes))

2424

In [16]:
note_dict = dict((note, number) for number, note in enumerate(pitches))

In [17]:
note_dict

{'0.1.3.7.81.0': 0,
 '0.1.5.80.25': 1,
 '0.1.5.80.5': 2,
 '0.1.5.81.0': 3,
 '0.1.50.75': 4,
 '0.1.60.5': 5,
 '0.1.62.5': 6,
 '0.10.0': 7,
 '0.2.3.5.70.5': 8,
 '0.2.4.70.5': 9,
 '0.2.4.71.0': 10,
 '0.2.40.5': 11,
 '0.2.5.70.5': 12,
 '0.2.50.5': 13,
 '0.2.52/3': 14,
 '0.2.60.25': 15,
 '0.2.60.5': 16,
 '0.2.61/3': 17,
 '0.2.62/3': 18,
 '0.2.64/3': 19,
 '0.2.70.5': 20,
 '0.2.70.75': 21,
 '0.2.71.0': 22,
 '0.2.71/3': 23,
 '0.20.0': 24,
 '0.20.25': 25,
 '0.20.5': 26,
 '0.20.75': 27,
 '0.21.0': 28,
 '0.21/3': 29,
 '0.22/3': 30,
 '0.3.50.5': 31,
 '0.3.51/3': 32,
 '0.3.52/3': 33,
 '0.3.6.91/3': 34,
 '0.3.6.94.0': 35,
 '0.3.60.5': 36,
 '0.3.62/3': 37,
 '0.3.70.25': 38,
 '0.3.70.5': 39,
 '0.3.70.75': 40,
 '0.3.71.0': 41,
 '0.3.71.5': 42,
 '0.3.71/3': 43,
 '0.30.25': 44,
 '0.30.5': 45,
 '0.30.75': 46,
 '0.31.0': 47,
 '0.31.5': 48,
 '0.31.75': 49,
 '0.31/3': 50,
 '0.33.0': 51,
 '0.34/3': 52,
 '0.4.52.0': 53,
 '0.4.60.5': 54,
 '0.4.60.75': 55,
 '0.4.70.25': 56,
 '0.4.70.5': 57,
 '0.4.70.75': 58,
 '0